In [161]:
def rank(similarity, q_pids, g_pids, topk=[1, 5, 10], get_mAP=True):
    max_rank = max(topk)
    if get_mAP:
        indices = torch.argsort(similarity, dim=1, descending=True)
    else:
        # acclerate sort with topk
        _, indices = torch.topk(
            similarity, k=max_rank, dim=1, largest=True, sorted=True
        )  # q * topk
    pred_labels = g_pids[indices]  # q * k
    matches = pred_labels.eq(q_pids.view(-1, 1))  # q * k

    all_cmc = matches[:, :max_rank].cumsum(1)
    all_cmc[all_cmc > 1] = 1
    all_cmc = all_cmc.float().mean(0) * 100
    all_cmc = all_cmc[topk - 1]

    if not get_mAP:
        return all_cmc, indices

    num_rel = matches.sum(1)  # q
    tmp_cmc = matches.cumsum(1)  # q * k
    tmp_cmc = [tmp_cmc[:, i] / (i + 1.0) for i in range(tmp_cmc.shape[1])]
    tmp_cmc = torch.stack(tmp_cmc, 1) * matches
    AP = tmp_cmc.sum(1) / num_rel  # q
    mAP = AP.mean() * 100
    return all_cmc, mAP, indices

In [162]:
import pickle as pkl
import numpy as np
args = dict()
args['k'] = 1
args['beta'] = 20

In [163]:
# Returns list of retrieved top k videos based on the sims matrix
def get_retrieved_videos(sims, k):
    argm = np.argsort(-sims, axis=1)
    topk = argm[:,:k].reshape(-1)
    retrieved_videos = np.unique(topk)
    return retrieved_videos

# Returns list of indices to normalize from sims based on videos
def get_index_to_normalize(sims, videos):
    argm = np.argsort(-sims, axis=1)[:,0]
    result = np.array(list(map(lambda x: x in videos, argm)))
    result = np.nonzero(result)
    return result

def qb_norm(train_test, test_test, args):
    k = args.get("k", 1)
    beta = args.get("beta", 20)
    retrieved_videos = get_retrieved_videos(train_test, k)
    test_test_normalized = test_test.copy()
    train_test = np.exp(train_test*beta)
    test_test = np.exp(test_test*beta)

    normalizing_sum = np.sum(train_test, axis=0)
    index_for_normalizing = get_index_to_normalize(test_test, retrieved_videos)
    test_test_normalized[index_for_normalizing, :] = \
        np.divide(test_test[index_for_normalizing, :], normalizing_sum)
    return test_test_normalized

In [88]:
# get data

In [182]:
train_text = np.load("train_text.npy")
test_text = np.load("test_text.npy")
test_image = np.load("test_image.npy")


In [183]:
train_test = np.matmul(train_text, test_image.T)
test_test = np.matmul(test_text, test_image.T)

In [184]:
train_test.shape,test_test.shape

((68126, 3074), (6156, 3074))

In [185]:
test_test_copy = test_test.copy()
test_test_copy

array([[0.79221576, 0.7817477 , 0.7255626 , ..., 0.11330012, 0.12920153,
        0.09065817],
       [0.81690973, 0.81889147, 0.7599835 , ..., 0.06901371, 0.08979727,
        0.05566901],
       [0.5925783 , 0.6066235 , 0.5384193 , ..., 0.12045822, 0.15623726,
        0.15992202],
       ...,
       [0.10873044, 0.15870313, 0.04203584, ..., 0.80582166, 0.79874676,
        0.62822354],
       [0.06607162, 0.09906943, 0.00132245, ..., 0.50819975, 0.5425391 ,
        0.6256979 ],
       [0.02234511, 0.03662704, 0.00830404, ..., 0.20784797, 0.25536132,
        0.5221216 ]], dtype=float32)

In [186]:
test_test_normalized = qb_norm(train_test, test_test, args)

In [187]:
np.nonzero(test_test_normalized != test_test_copy)

(array([   0,    0,    0, ..., 6155, 6155, 6155]),
 array([   0,    1,    2, ..., 3071, 3072, 3073]))

In [188]:
import torch
topk = torch.tensor([1,5,10])
text_pid = np.load("text_pid.npy")
image_pid = np.load("image_pid.npy")
t2i_cmc, _ = rank(torch.tensor(test_test_normalized), torch.tensor(text_pid), torch.tensor(image_pid), topk, get_mAP=False)

In [190]:
for i in range(10,20,1):
    args['beta'] = i
    print(i)
    test_test_normalized = qb_norm(train_test, test_test, args)
    t2i_cmc, _ = rank(torch.tensor(test_test_normalized), torch.tensor(text_pid), torch.tensor(image_pid), topk, get_mAP=False)
    print(t2i_cmc)

10
tensor([59.2593, 77.2904, 84.6004])
11
tensor([59.4217, 77.5666, 84.7466])
12
tensor([59.5517, 77.6803, 84.8765])
13
tensor([59.6816, 77.7778, 84.9415])
14
tensor([59.6979, 77.8752, 84.9253])
15
tensor([59.6654, 77.9077, 85.0065])
16
tensor([59.7303, 77.9890, 85.0715])
17
tensor([59.6654, 78.0052, 84.8928])
18
tensor([59.7303, 77.9727, 84.9578])
19
tensor([59.5679, 77.8752, 85.1040])


In [171]:
t2i_cmc

tensor([57.2612, 76.4457, 84.3405])

In [153]:
test_test = test_test_copy.copy()
test_test

array([[ 0.80003065,  0.77750695,  0.7724082 , ...,  0.152077  ,
         0.16015589,  0.07175662],
       [ 0.78522956,  0.7851329 ,  0.7709395 , ...,  0.15204026,
         0.15973394,  0.05422914],
       [ 0.48401484,  0.52995706,  0.43170282, ...,  0.10086777,
         0.1578648 ,  0.11296237],
       ...,
       [ 0.21264191,  0.2570559 ,  0.1500122 , ...,  0.70756483,
         0.7190536 ,  0.646068  ],
       [ 0.10210612,  0.12128984, -0.00372251, ...,  0.3991889 ,
         0.48524168,  0.5229347 ],
       [-0.12080702, -0.10585881, -0.18819067, ...,  0.14036119,
         0.21350087,  0.35388976]], dtype=float32)

In [149]:
train_test

array([[ 0.07206351,  0.06216157,  0.06284328, ...,  0.25377652,
         0.20778054,  0.18290631],
       [ 0.0627052 ,  0.07563372,  0.09879597, ...,  0.12553865,
         0.10225028,  0.11241295],
       [ 0.00257592,  0.01570105,  0.03701405, ...,  0.2815793 ,
         0.3008128 ,  0.16156209],
       ...,
       [ 0.20475474,  0.14967012,  0.23735371, ..., -0.12485246,
        -0.10914449, -0.16367406],
       [ 0.13384305,  0.10074963,  0.10405405, ..., -0.09958716,
        -0.08937244, -0.18648036],
       [ 0.22390501,  0.20834157,  0.19274658, ...,  0.01956081,
        -0.00192396, -0.0893784 ]], dtype=float32)